In [3]:
import pypsa
import pandas as pd
import yaml
import re
import xarray as xr

In [4]:
def assign_carriers(n):

    if "Load" in n.carriers.index:
        n.carriers = n.carriers.drop("Load")

    if "carrier" not in n.lines:
        n.lines["carrier"] = "AC"

    if n.links.empty:
        n.links["carrier"] = pd.Series(dtype=str)

    config = {
        "AC": {"color": "rosybrown", "nice_name": "HVAC Line"},
        "DC": {"color": "darkseagreen", "nice_name": "HVDC Link"},
    }

    for c in ["AC", "DC"]:
        if c in n.carriers.index:
            continue
        n.carriers = n.carriers.append(pd.Series(config[c], name=c))

def aggregate_costs(n, existing_only=False, by_carrier=True):

    assign_carriers(n)
    
    components = dict(
        Link=("p_nom", "p0"),
        Generator=("p_nom", "p"),
        StorageUnit=("p_nom", "p"),
        Store=("e_nom", "p"),
        Line=("s_nom", None),
    )

    costs = {}
    for c in n.iterate_components(components.keys()):
        p_nom, p_attr = components[c.name]
        if c.df.empty:
            continue
        if not existing_only:
            p_nom += "_opt"
        costs[(c.list_name, "capital")] = (
            (c.df[p_nom] * c.df.capital_cost).groupby(c.df.carrier).sum()
        )
        if p_attr is not None:
            p = c.pnl[p_attr].multiply(n.snapshot_weightings, axis=0).sum()
            if c.name == "StorageUnit":
                p = p.loc[p > 0]
            costs[(c.list_name, "marginal")] = (
                (p * c.df.marginal_cost).groupby(c.df.carrier).sum()
            )
    costs = pd.concat(costs) / 1e9  # bn EUR/a

    if by_carrier:
        costs = costs.groupby(level=2).sum()

    return costs

def get_stats(n):
    stats = pd.concat(
        [
            n.generators.groupby("carrier").p_nom_opt.sum() / 1e3,  # GW
            n.storage_units.groupby("carrier").p_nom_opt.sum() / 1e3,  # GW
            pd.Series(
                {"links": n.links.eval("length * (p_nom_opt - p_nom) / 1e6").sum()}
            ),  # TWkm
            pd.Series(
                {"lines": n.lines.eval("length * (s_nom_opt - s_nom) / 1e6").sum()}
            ),  # TWkm
        ]
    )
    
    stats["tsc"] = aggregate_costs(n).sum()

    if "load" in stats.index:
        stats.drop("load", inplace=True)

    return stats

In [5]:
with open("config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [7]:
# overwrite to subset
config["scenario"]["opts"] = [
    "3H-solar+1.0-onwind+1.5-offwind+1.0-H2+1.0-battery+0.5",
    "3H-solar+1.0-onwind+1.5-offwind+1.5-H2+0.5-battery+1.5",
    "3H-solar+1.5-onwind+0.5-offwind+0.5-H2+0.5-battery+1.5"
]

In [8]:
scenarios_opts = config["scenario"]["opts"]

In [9]:
ns = {opts: pypsa.Network(f"networks/elec_s_100_ec_lcopt_{opts}.nc") for opts in scenarios_opts}

INFO:pypsa.io:Imported network elec_s_100_ec_lcopt_3H-solar+1.0-onwind+1.5-offwind+1.0-H2+1.0-battery+0.5.nc has buses, carriers, generators, lines, links, loads, storage_units
INFO:pypsa.io:Imported network elec_s_100_ec_lcopt_3H-solar+1.0-onwind+1.5-offwind+1.5-H2+0.5-battery+1.5.nc has buses, carriers, generators, lines, links, loads, storage_units
INFO:pypsa.io:Imported network elec_s_100_ec_lcopt_3H-solar+1.5-onwind+0.5-offwind+0.5-H2+0.5-battery+1.5.nc has buses, carriers, generators, lines, links, loads, storage_units


In [14]:
df = pd.concat({opts: get_stats(ns[opts]) for opts in scenarios_opts}, axis=1)

In [15]:
df

,3H-solar+1.0-onwind+1.5-offwind+1.0-H2+1.0-battery+0.5,3H-solar+1.0-onwind+1.5-offwind+1.5-H2+0.5-battery+1.5,3H-solar+1.5-onwind+0.5-offwind+0.5-H2+0.5-battery+1.5
offwind-ac,285.870438,253.908769,0.005999
offwind-dc,225.127161,290.121320,0.004785
onwind,89.849930,31.381633,1387.550558
ror,34.503917,34.503917,34.503917
solar,445.626110,385.749386,32.273482
H2,90.963706,181.477896,165.958901
PHS,54.593581,54.593581,54.593581
battery,83.984855,0.022561,0.011280
hydro,99.562201,99.562201,99.562201
links,31.268197,23.097868,28.174029


## Building MultiIndex from `opts`

In [13]:
def parse(opts):
    data = {}
    for o in opts.split("-"):
        s = o.split("+")
        if len(s) > 1:
            carrier = s[0]
            factor = float(s[1])
            data[carrier] = factor
    return pd.Series(data)

In [14]:
idx_map = pd.concat({opts: parse(opts) for opts in scenarios_opts}, axis=1).fillna(1.)

In [15]:
df.columns = pd.MultiIndex.from_tuples([idx_map[i] for i in df.columns],
                                       names=[f"{i}-cost" for i in idx_map.index])

In [16]:
df.to_csv("capacities.csv")

solar-cost             1.0                         1.5
onwind-cost            1.5                         0.5
offwind-cost           1.0           1.5           0.5
H2-cost                1.0           0.5           0.5
battery-cost           0.5           1.5           1.5
offwind-ac    2.858704e+02  2.539088e+02  5.999401e-03
offwind-dc    2.251272e+02  2.901213e+02  4.785418e-03
onwind        8.984993e+01  3.138163e+01  1.387551e+03
ror           3.450392e+01  3.450392e+01  3.450392e+01
solar         4.456261e+02  3.857494e+02  3.227348e+01
H2            9.096371e+01  1.814779e+02  1.659589e+02
PHS           5.459358e+01  5.459358e+01  5.459358e+01
battery       8.398486e+01  2.256058e-02  1.128000e-02
hydro         9.956220e+01  9.956220e+01  9.956220e+01
links         3.126820e+01  2.309787e+01  2.817403e+01
lines         3.141886e+02  2.901117e+02  2.911575e+02
objective     2.037410e+11  1.911724e+11  1.257277e+11

## Convert to `xr.Dataset`

In [14]:
ds = xr.Dataset.from_dataframe(df.T, sparse=False)

In [278]:
ds.to_netcdf("capacities.nc")

## Building multiindex from dictionary

params = dict(
    solar=[0.5,1.0],
    onwind=[1.0],
    offwind=[1.0],
    H2=[1.0],
    battery=[1.0])

pd.MultiIndex.from_product(params.values(), names=params.keys())

In [10]:
n = ns["3H-solar+1.5-onwind+0.5-offwind+0.5-H2+0.5-battery+1.5"]

In [15]:
[c.split("-")[0] for c in n.carriers.index]

['onwind',
 'offwind',
 'offwind',
 'solar',
 'PHS',
 'hydro',
 'ror',
 'H2',
 'battery']

True